# Introduction
### Project: Creating NBA statistic database & predicting win totals based on individual players
- Name: Alex Sanchez
- NetID: aes341

# Step 1:
## Prepare and process data
Sourced from Kaggle dataset, linked:
- https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats/data?select=Team+Summaries.csv
- https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats/data?select=Player+Per+Game.csv

In [1]:
# will read into pandas dataframe, edit as needed, and then place into database.
import pandas as pd
import sqlite3

In [2]:
# Load player per game dataset
player_stats = pd.read_csv('Player Per Game.csv')

# Load the team summaries dataset
team_summaries = pd.read_csv('Team Summaries.csv')

# I am planning on using only some of the columns in these datasets, not all of them are relevant to me.
# The rest will just be filtered out as follows:
player_columns = [
    "player_id", "player", "g", "gs", "pos", "fg_percent", "x3p_percent",
    "e_fg_percent", "trb_per_game", "ast_per_game", "stl_per_game",
    "blk_per_game", "tov_per_game", "pts_per_game", "tm", "season"
]
filtered_players = player_stats[player_columns].copy() # only use the columns that i want

# Some players, commonly bigs, dont shoot 3 pointers very often. Some never will in a season, so their x3p_precent
# will be NA. when I remove NA columns, i dont want these players removed.
# The same is the case for blocks per game, and steals per game, and assits per game
# Players who average 0 points or rebounds per game likely are not playing any signifant time, so they can be ignored

filtered_players['x3p_percent'] = filtered_players['x3p_percent'].fillna(0)
filtered_players['tov_per_game'] = filtered_players['tov_per_game'].fillna(0)
filtered_players['blk_per_game'] = filtered_players['blk_per_game'].fillna(0)
filtered_players['stl_per_game'] = filtered_players['stl_per_game'].fillna(0)

# Select columns for the team summaries
team_columns = ["abbreviation", "season", "w"]

filtered_teams = team_summaries[team_columns]

In [3]:
# show head of filtered teams
filtered_teams.head()

,abbreviation,season,w
0,ATL,2025,10.0
1,BOS,2025,16.0
2,BRK,2025,9.0
3,CHI,2025,8.0
4,CHO,2025,6.0


In [4]:
filtered_players[filtered_players['tm'] == 'CLE'].head(20)

#Some other manual changes =>
# in this dataset, OKC is not written to have a point gaurd, as Shai Gilgeous-Alexander
# is listed as a SG and PG technically, although he serves as their starting point gaurd.

filtered_players.loc[filtered_players['player_id'] == 4669, 'pos'] = 'PG' # this will come up again for current year predictions

# Similarly, Cleveland has been starting Dean Wade as their SF
filtered_players.loc[filtered_players['player_id'] == 4710, 'pos'] = "SF"

# Lastly, Memphis has been starting Jaylen Wells as their SF
filtered_players.loc[filtered_players['player_id'] == 5241, 'pos'] = "SF"

# these conflicts only really exist because this season is very young. Doing this for previous seasons
# is overall insignificant

In [5]:
# remove unneeded players
print(len(filtered_players))
filtered_players.dropna(inplace=True)
print(len(filtered_players))
# go from 32358 players -> 23588. Most of the NA items are just from old NBA, where statistics were less well kept.

32358
23588


In [6]:
# show head of filtered_players
filtered_players.sample(100)

,player_id,player,g,gs,pos,fg_percent,x3p_percent,e_fg_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,tm,season
21816,2222,Benoit Benjamin,79,37.0,C,0.490,0.333,0.491,7.6,1.0,0.8,2.6,1.8,11.1,LAC,1986
7921,3506,Beno Udrih,27,9.0,PG,0.408,0.396,0.452,2.3,6.1,0.9,0.0,2.0,10.2,ORL,2013
3689,3880,James Harden,68,68.0,SG,0.444,0.355,0.543,6.6,7.5,1.8,0.9,4.5,34.3,HOU,2020
22735,1595,Len Elmore,65,5.0,PF,0.408,0.000,0.408,2.5,0.5,0.4,0.5,0.7,2.4,NYK,1984
5345,4448,Skal Labissière,60,28.0,PF,0.448,0.353,0.467,4.8,1.2,0.4,0.8,1.2,8.7,SAC,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056,3530,Jameer Nelson,7,0.0,PG,0.282,0.071,0.295,1.1,3.3,0.6,0.1,1.6,3.7,DET,2018
14561,2552,Tim Hardaway,68,16.0,PG,0.365,0.347,0.464,1.8,4.1,0.8,0.1,1.6,9.6,TOT,2002
19197,2401,Winston Garland,66,4.0,PG,0.443,0.462,0.452,1.6,2.1,0.6,0.1,1.0,5.9,HOU,1993
21338,2092,Rod Higgins,68,67.0,SF,0.526,0.487,0.539,4.3,2.8,1.0,0.5,1.6,15.5,GSW,1988


In [7]:
# To prove that the tm and abbriviations are the same teams:
team_set = set(filtered_players['tm'])
abbr_set = set(filtered_teams['abbreviation'])
print("Unmatched teams:", team_set - abbr_set)

# The unmatched team "TOT" is just from BasketballRefrence.com
# It is for players who play on more than 1 team per year, and stands for their total performance on every team for that season
# These are unneeded for my purposes, so I will remove them
filtered_players = filtered_players[filtered_players['tm'] != 'TOT'] # I will also use this for 2024 test so I want these dif names

Unmatched teams: {'TOT'}


In [8]:
# I want to modify filtered_players to have one more column, for the amount of wins they (their team)
# had in the previous year. We can do this by merging the season & abbreviation columns in the two dataframes

# Rename 'abbreviation' to 'tm' in team_summaries for consistency
team_summaries.rename(columns={"abbreviation": "tm"}, inplace=True)

# Merge player_stats with team_summaries on 'tm' and 'season'
player_stats_with_wins = filtered_players.merge(
    team_summaries[["tm", "season", "w"]],
    on=["tm", "season"],
    how="left"  # Use left join to keep all player_stats rows
)

# now we can look at the results.
player_stats_with_wins.head(10)

,player_id,player,g,gs,pos,fg_percent,x3p_percent,e_fg_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,tm,season,w
0,5025,A.J. Green,18,3.0,SG,0.490,0.478,0.710,2.0,0.9,0.6,0.1,0.5,8.2,MIL,2025,10.0
1,5210,AJ Johnson,4,0.0,SG,0.333,0.000,0.333,0.3,0.3,0.0,0.0,0.3,0.5,MIL,2025,10.0
2,4219,Aaron Gordon,7,7.0,PF,0.529,0.545,0.614,6.7,3.1,0.7,0.3,1.3,15.4,DEN,2025,10.0
3,4582,Aaron Holiday,11,0.0,PG,0.452,0.391,0.597,0.8,1.3,0.1,0.1,0.3,3.8,HOU,2025,14.0
4,4805,Aaron Nesmith,6,6.0,SF,0.528,0.545,0.611,4.0,1.0,0.7,0.3,0.8,9.2,IND,2025,9.0
5,4900,Aaron Wiggins,19,8.0,SG,0.447,0.420,0.543,3.8,1.5,0.8,0.3,0.8,9.2,OKC,2025,15.0
6,5109,Adam Flagler,9,0.0,SG,0.267,0.286,0.400,0.6,0.0,0.2,0.0,0.0,1.3,OKC,2025,15.0
7,5110,Adama Sanogo,3,0.0,C,0.600,0.000,0.600,0.7,0.3,0.0,0.0,0.3,2.0,CHI,2025,8.0
8,5211,Adem Bona,12,0.0,PF,0.750,0.000,0.750,1.7,0.0,0.0,0.5,0.3,1.3,PHI,2025,4.0
9,5212,Ajay Mitchell,19,0.0,SG,0.477,0.423,0.540,1.5,1.6,0.9,0.1,0.8,5.5,OKC,2025,15.0


In [9]:
# I also want to group teams by their starting 5 players, as will be used later in predictive modeling.
# Sort by team, season, and games started (descending)

def balanced_lineup(group):
    # Define required positions and initialize an empty lineup
    positions = {"PG", "SG", "SF", "PF", "C"}
    lineup = []
    for pos in positions: # do once for each position
        # Filter for players matching the position
        players = group[group["pos"] == pos]
        
        if players.empty: # if there are no players of this position on this team, just put in placeholder (remove later)
            placeholder = pd.DataFrame([{
                "tm": None,
                "season": None,
                "pos": None, 
                "gs": 0, 
                "player": None,
            }])
            lineup.append(placeholder)
        else:    
            # Select the top player by games started 'gs'
            top_player = players.sort_values(by="gs", ascending=False).head(1)
            lineup.append(top_player)
    # Concatenate the selected players into a single DataFrame
    return pd.concat(lineup) # return to apply function, for each team passed to it

# Apply the function to each team-season group

ranked_players = (
    player_stats_with_wins
    .groupby(["tm", "season"], group_keys=True)  # i want group keys
    .apply(balanced_lineup, include_groups=False)   # get warnings without include_groups=False
)


starters = ranked_players.drop(columns=['tm', 'season']) # remove these so that i can reset_index() and put them back
starters.columns

Index(['player_id', 'player', 'g', 'gs', 'pos', 'fg_percent', 'x3p_percent',
       'e_fg_percent', 'trb_per_game', 'ast_per_game', 'stl_per_game',
       'blk_per_game', 'tov_per_game', 'pts_per_game', 'w'],
      dtype='object')

In [10]:
# We can see that for each team, we can access the 5 positional players that started the most games
# I will be refering to these players as the "starting 5" for any specific team in some specific year
# for loading in the DB, I run into issues with the team name and season not actually included as seasons.
# this code explicitely adds those columns back. 

ranked_players = starters.reset_index()
ranked_players = ranked_players.drop(columns="level_2") # i am not really sure where this comes from, but it causes issues
player_stats_with_wins.head(75)
# now as we can see, tm and season are added back as columns. This solves my db issues.

,player_id,player,g,gs,pos,fg_percent,x3p_percent,e_fg_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,tm,season,w
0,5025,A.J. Green,18,3.0,SG,0.490,0.478,0.710,2.0,0.9,0.6,0.1,0.5,8.2,MIL,2025,10.0
1,5210,AJ Johnson,4,0.0,SG,0.333,0.000,0.333,0.3,0.3,0.0,0.0,0.3,0.5,MIL,2025,10.0
2,4219,Aaron Gordon,7,7.0,PF,0.529,0.545,0.614,6.7,3.1,0.7,0.3,1.3,15.4,DEN,2025,10.0
3,4582,Aaron Holiday,11,0.0,PG,0.452,0.391,0.597,0.8,1.3,0.1,0.1,0.3,3.8,HOU,2025,14.0
4,4805,Aaron Nesmith,6,6.0,SF,0.528,0.545,0.611,4.0,1.0,0.7,0.3,0.8,9.2,IND,2025,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,4382,Caris LeVert,14,0.0,SG,0.528,0.471,0.639,2.8,4.3,0.9,0.5,1.1,11.5,CLE,2025,17.0
71,5223,Carlton Carrington,18,12.0,PG,0.401,0.352,0.489,3.9,4.4,1.2,0.3,1.9,8.8,WAS,2025,2.0
72,5124,Cason Wallace,19,9.0,SG,0.413,0.259,0.471,3.3,2.1,1.7,0.7,0.4,6.2,OKC,2025,15.0
73,4917,Charles Bassey,11,0.0,C,0.577,0.000,0.577,3.1,0.4,0.5,1.1,0.5,3.2,SAS,2025,10.0


# Part 2: Database

- I will connect to a database and load my ranked_players info (for later), as well as the filtered_players set, for more detailed querying

In [11]:
db = sqlite3.connect('nba_stats.db')
cursor = db.cursor()
# note that the order of columns is different in the player_stats_with_wins and the ranked_players dataframes because of the way i did things. [again, ranked_players refers to the starters]
# create the table for player stats and for starters only. ensure that the table is dropped if already exists (replace it)
cursor.execute("DROP TABLE IF EXISTS player_stats;")
cursor.execute('''
CREATE TABLE player_stats (
    player_id INTEGER,
    player TEXT,
    g FLOAT,
    gs FLOAT,
    pos TEXT,
    fg_percent FLOAT,
    x3p_percent FLOAT,
    e_fg_percent FLOAT,
    trb_per_game FLOAT,
    ast_per_game FLOAT,
    stl_per_game FLOAT,
    blk_per_game FLOAT,
    tov_per_game FLOAT,
    pts_per_game FLOAT,
    tm TEXT,
    season INTEGER,
    w FLOAT
)
''')
db.commit() # make the table.

cursor.execute("DROP TABLE IF EXISTS starters_only;")
cursor.execute('''
CREATE TABLE starters_only (
    tm TEXT,
    season INTEGER,
    player_id INTEGER,
    player TEXT,
    g FLOAT,
    gs FLOAT,
    pos TEXT,
    fg_percent FLOAT,
    x3p_percent FLOAT,
    e_fg_percent FLOAT,
    trb_per_game FLOAT,
    ast_per_game FLOAT,
    stl_per_game FLOAT,
    blk_per_game FLOAT,
    tov_per_game FLOAT,
    pts_per_game FLOAT,
    w FLOAT
)
''')
db.commit() # make the table.
for _, row in player_stats_with_wins.iterrows():
    cursor.execute('''
    INSERT INTO player_stats (
        player_id, player, g, gs, pos,
        fg_percent, x3p_percent, e_fg_percent, trb_per_game,
        ast_per_game, stl_per_game, blk_per_game,
        tov_per_game, pts_per_game, tm, season, w
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row))
db.commit() # insert every row

for _, row in ranked_players.iterrows():
    cursor.execute('''
    INSERT INTO starters_only (
        tm, season, player_id, player, g, gs, pos,
        fg_percent, x3p_percent, e_fg_percent, trb_per_game,
        ast_per_game, stl_per_game, blk_per_game,
        tov_per_game, pts_per_game, w
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row))
db.commit() # do the same for ranked_players (the starters only)


# show that this works, read into a pandas df
pd.read_sql_query("SELECT * FROM player_stats WHERE tm = 'WAS' AND season = 2025;", db)

,player_id,player,g,gs,pos,fg_percent,x3p_percent,e_fg_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,tm,season,w
0,5215,Alex Sarr,18.0,18.0,C,0.376,0.244,0.429,6.1,2.1,0.7,1.8,1.3,10.6,WAS,2025,2.0
1,4809,Anthony Gill,13.0,0.0,PF,0.588,0.500,0.676,0.9,0.4,0.2,0.1,0.1,2.3,WAS,2025,2.0
2,5119,Bilal Coulibaly,17.0,17.0,SF,0.482,0.309,0.534,5.4,2.8,1.3,0.6,1.9,12.6,WAS,2025,2.0
3,5223,Carlton Carrington,18.0,12.0,PG,0.401,0.352,0.489,3.9,4.4,1.2,0.3,1.9,8.8,WAS,2025,2.0
4,4920,Corey Kispert,18.0,0.0,SF,0.417,0.315,0.514,3.5,1.4,0.7,0.1,1.2,11.3,WAS,2025,2.0
5,4953,Jared Butler,13.0,0.0,SG,0.465,0.167,0.477,0.6,2.2,0.5,0.2,0.5,4.3,WAS,2025,2.0
6,5066,Johnny Davis,13.0,0.0,SG,0.483,0.500,0.534,1.2,0.2,0.3,0.1,0.1,2.6,WAS,2025,2.0
7,4100,Jonas Valančiūnas,18.0,3.0,C,0.581,0.250,0.588,7.6,2.2,0.4,0.7,1.7,12.0,WAS,2025,2.0
8,4737,Jordan Poole,16.0,16.0,SG,0.448,0.421,0.550,2.3,4.9,1.9,0.5,3.4,21.0,WAS,2025,2.0
9,4534,Kyle Kuzma,12.0,12.0,PF,0.420,0.276,0.464,5.6,1.9,0.4,0.3,2.4,15.8,WAS,2025,2.0


## What are some questions we can ask?

In [12]:
# Now we have our database set up. With this in place, we can easily query for 
# any stat related question! (in the bounds of what I have saved)
# For example, I can answer the question:
# What players have averaged 35 points per game?
query = "SELECT * FROM player_stats WHERE pts_per_game >= 35;"
players_with_35_ppg = pd.read_sql_query(query, db)

players_with_35_ppg
# Only 3 players have ever done this! And Michael Jordan did it twice! 

,player_id,player,g,gs,pos,fg_percent,x3p_percent,e_fg_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,tm,season,w
0,3880,James Harden,78.0,78.0,PG,0.442,0.368,0.541,6.6,7.5,2.0,0.7,5.0,36.1,HOU,2019,53.0
1,3000,Kobe Bryant,80.0,80.0,SG,0.450,0.347,0.491,5.3,4.5,1.8,0.4,3.1,35.4,LAL,2006,45.0
2,2193,Michael Jordan,82.0,82.0,SG,0.535,0.132,0.537,5.5,5.9,3.2,1.6,3.1,35.0,CHI,1988,50.0
3,2193,Michael Jordan,82.0,82.0,SG,0.482,0.182,0.484,5.2,4.6,2.9,1.5,3.3,37.1,CHI,1987,40.0


In [13]:
# What players have averaged 30 points per game and 40% from 3 pointers?

query = "SELECT player, x3p_percent, season, tm FROM player_stats WHERE pts_per_game >= 30 AND x3p_percent >= .40;"
players_30ppg_40p3 = pd.read_sql_query(query, db)

players_30ppg_40p3
# Another interesting result! Again only 3 players have ever done this, and Stephen Curry did so twice

,player,x3p_percent,season,tm
0,Stephen Curry,0.421,2021,GSW
1,Damian Lillard,0.401,2020,POR
2,Stephen Curry,0.454,2016,GSW
3,Michael Jordan,0.427,1996,CHI


In [14]:
# What players have averaged over 20 points, 10 rebounds, and 10 assists per game for a season?
query = "SELECT * FROM player_stats WHERE pts_per_game >= 20 AND trb_per_game >=10 AND ast_per_game >=10;"
trip_double = pd.read_sql_query(query, db)

trip_double

,player_id,player,g,gs,pos,fg_percent,x3p_percent,e_fg_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game,tm,season,w
0,4352,Nikola Jokić,14.0,14.0,C,0.568,0.534,0.625,13.1,10.6,1.5,0.9,3.6,29.7,DEN,2025,10.0
1,3849,Russell Westbrook,65.0,65.0,PG,0.439,0.315,0.474,11.5,11.7,1.4,0.4,4.8,22.2,WAS,2021,34.0
2,3849,Russell Westbrook,73.0,73.0,PG,0.428,0.290,0.468,11.1,10.7,1.9,0.5,4.5,22.9,OKC,2019,49.0
3,3849,Russell Westbrook,80.0,80.0,PG,0.449,0.298,0.477,10.1,10.3,1.8,0.3,4.8,25.4,OKC,2018,48.0
4,3849,Russell Westbrook,81.0,81.0,PG,0.425,0.343,0.476,10.7,10.4,1.6,0.4,5.4,31.6,OKC,2017,47.0


In [16]:
# We can use the starters_only table too for some interesting informtion! 
# We can check which seasons in history contained the highest average scoring from all starters on some team!

query = "SELECT tm, season, AVG(pts_per_game) AS avg_starter_points FROM starters_only GROUP BY tm, season ORDER BY avg_starter_points DESC LIMIT 10;"
teams_most_pts_starters = pd.read_sql_query(query, db)

teams_most_pts_starters

# This also exemplifies the idea that in the "modern NBA", scoring is higher than ever
# Most teams are recent, but still several teams in the 80s had many high scoring starters!

,tm,season,avg_starter_points
0,PHI,2024,20.86
1,BOS,1987,19.98
2,NYK,2025,19.80
3,GSW,2019,19.70
4,DEN,2025,19.60
5,BOS,2024,19.54
6,POR,2023,19.32
7,BOS,1988,19.02
8,BOS,1985,18.94
9,SAC,2005,18.88


In [17]:
# We can also quickly analyze some historical trends, such as the number of of players who have
# averaged 30 points in a season by season (can tell which seasons had more people scoring more points)
query = "SELECT season, COUNT(*) AS players_avg_25 FROM player_stats WHERE pts_per_game >= 25 GROUP BY season ORDER BY players_avg_25 DESC LIMIT 6;"
more_people_more_points = pd.read_sql_query(query, db)

more_people_more_points

# Again, shows recent seasons have more scoring than ever! Very interesting!

,season,players_avg_25
0,2023,23
1,2024,18
2,2021,17
3,2025,15
4,2022,14
5,2020,14


In [18]:
# We can do the same thing, but for blocks for example, to see 
# how often many people averaged 10+ rebounds:

query = "SELECT season, COUNT(*) AS num_players FROM player_stats WHERE trb_per_game >= 10 GROUP BY season ORDER BY num_players DESC LIMIT 6;"
more_people_more_reb = pd.read_sql_query(query, db)

more_people_more_reb

# Again, shows recent seasons have more scoring than ever! Very interesting!
# This one is more varied, and is harder to really come to any conclusions.

,season,num_players
0,1994,23
1,2019,19
2,2022,18
3,2021,17
4,2020,17
5,2017,16


In [19]:
# How about for assists? are people passing the ball now more than ever?

query = "SELECT season, COUNT(*) AS num_players FROM player_stats WHERE ast_per_game >= 8 GROUP BY season ORDER BY num_players DESC LIMIT 6;"
more_people_more_ast = pd.read_sql_query(query, db)

more_people_more_ast

# No! The 90s had the most high assist players by far! An interesting historical trend!

,season,num_players
0,1991,15
1,1998,11
2,1997,11
3,1990,11
4,1988,11
5,1986,11


# Part 3: Predition Modeling
- I want to predict a teams wins, based on a teams' starters and how they preformed in the previous year
- To maximize the importance of each stat, I want to seperate positional stats in how they affect a team's performance.

In [31]:
# Some new imports are necessary:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

In [32]:
ranked_players.dropna(inplace=True)
cleaned_1 = ranked_players # I will also use this for 2024 test so I want these dif names
filtered_df = cleaned_1[cleaned_1['season'] != 2024] # I want to use this to test, so I wont train on it
filtered_df = filtered_df[filtered_df['season'] != 2025] # I want to use this to test, so I wont train on it
# these teams get in the way


pivoted_df = filtered_df.pivot(index=['tm', 'season'], columns='pos', values=['g','gs','pos', 'fg_percent','x3p_percent',
                                                                              'e_fg_percent','trb_per_game','ast_per_game',
                                                                              'stl_per_game','blk_per_game','tov_per_game',
                                                                              'pts_per_game'])

# Flatten the multi-index columns
pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns]

pivoted_df['w'] = filtered_df.groupby(['tm', 'season'])['w'].first().values

pivoted_df.dropna(inplace=True)

# There are sparse cases where my method does not really work for finding the 5 starters, 
# where a team will have duplicate positions starting more games than another (ie, 2 SG that start more games than 1 center)
# I tried to limit these but there are still some that get through. Those will just be removed.
pivoted_df
# we can see that still, 1172 rows (which means 1172 examinable season team performances) exist. This is good enough.

g_C  g_PF  g_PG  g_SF  g_SG  gs_C gs_PF gs_PG gs_SF gs_SG  ...  \
tm  season                                                              ...   
ATL 1982    79.0  61.0  82.0  70.0  68.0  39.0  58.0  82.0  51.0  57.0  ...   
    1983    80.0  77.0  49.0  82.0  61.0  80.0  76.0  49.0  82.0  57.0  ...   
    1984    77.0  73.0  75.0  81.0  67.0  76.0  72.0  72.0  81.0  43.0  ...   
    1985    70.0  74.0  69.0  81.0  73.0  60.0  53.0  58.0  81.0  66.0  ...   
    1986    74.0  82.0  53.0  78.0  81.0  61.0  59.0  50.0  78.0  79.0  ...   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
WSB 1993    49.0  81.0  70.0  72.0  69.0  48.0  81.0  70.0  72.0  33.0  ...   
    1994    69.0  78.0  70.0  75.0  60.0  52.0  78.0  67.0  69.0  59.0  ...   
    1995    73.0  54.0  62.0  62.0  78.0  58.0  52.0  62.0  13.0  71.0  ...   
    1996    76.0  81.0  81.0  70.0  55.0  76.0  81.0  50.0  70.0  26.0  ...   
    1997    73.0  72.0  82.0  82.0  79.0  69.0  72.0  81.0  82.0  79.0  ...   

           tov_per_game_PF tov_per_game_PG tov_per_game_SF tov_per_game_SG  \
tm  season                                                                   
ATL 1982               3.0             1.8             2.5             2.7   
    1983               3.2             2.6             2.2             2.6   
    1984               2.8             1.8             2.7             2.6   
    1985               1.8             2.6             2.8             3.3   
    1986               2.2             2.7             3.2             1.4   
...                    ...             ...             ...             ...   
WSB 1993               2.8             2.5             1.3             1.5   
    1994               3.2             2.4             2.0             2.0   
    1995               3.1             2.8             0.9             1.9   
    1996               3.7             1.9             1.8             1.0   
    1997               3.2             3.3             3.0             1.2   

           pts_per_game_C pts_per_game_PF pts_per_game_PG pts_per_game_SF  \
tm  season                                                                  
ATL 1982              6.1            18.6            10.5            18.5   
    1983              7.8            19.0            12.6            17.5   
    1984              8.6            18.9            12.3            21.6   
    1985              6.3             9.8            14.1            27.4   
    1986              5.6            12.3            11.5            30.3   
...                   ...             ...             ...             ...   
WSB 1993             17.4            14.7            14.8            18.6   
    1994              6.6            17.1            12.1            18.2   
    1995             10.0            20.1            13.0             3.9   
    1996             14.5            22.1            10.0            15.1   
    1997             10.6            20.1            17.2            19.1   

           pts_per_game_SG     w  
tm  season                        
ATL 1982              17.8  42.0  
    1983              16.0  43.0  
    1984              13.2  40.0  
    1985              16.3  34.0  
    1986              12.9  50.0  
...                    ...   ...  
WSB 1993               9.3  22.0  
    1994              18.2  24.0  
    1995              16.6  21.0  
    1996               8.6  39.0  
    1997              10.6  44.0  

[1184 rows x 61 columns]

In [33]:
# Set up X and Y for prediction model

X = pivoted_df.drop(columns=['w', 'pos_PG', 'pos_SG', 'pos_SF', 'pos_PF', 'pos_C'])
# all independant variables are X
# we want to predcit wins, so:
y= pivoted_df['w'] # only wins
X

g_C  g_PF  g_PG  g_SF  g_SG  gs_C gs_PF gs_PG gs_SF gs_SG  ...  \
tm  season                                                              ...   
ATL 1982    79.0  61.0  82.0  70.0  68.0  39.0  58.0  82.0  51.0  57.0  ...   
    1983    80.0  77.0  49.0  82.0  61.0  80.0  76.0  49.0  82.0  57.0  ...   
    1984    77.0  73.0  75.0  81.0  67.0  76.0  72.0  72.0  81.0  43.0  ...   
    1985    70.0  74.0  69.0  81.0  73.0  60.0  53.0  58.0  81.0  66.0  ...   
    1986    74.0  82.0  53.0  78.0  81.0  61.0  59.0  50.0  78.0  79.0  ...   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
WSB 1993    49.0  81.0  70.0  72.0  69.0  48.0  81.0  70.0  72.0  33.0  ...   
    1994    69.0  78.0  70.0  75.0  60.0  52.0  78.0  67.0  69.0  59.0  ...   
    1995    73.0  54.0  62.0  62.0  78.0  58.0  52.0  62.0  13.0  71.0  ...   
    1996    76.0  81.0  81.0  70.0  55.0  76.0  81.0  50.0  70.0  26.0  ...   
    1997    73.0  72.0  82.0  82.0  79.0  69.0  72.0  81.0  82.0  79.0  ...   

           tov_per_game_C tov_per_game_PF tov_per_game_PG tov_per_game_SF  \
tm  season                                                                  
ATL 1982              1.0             3.0             1.8             2.5   
    1983              1.2             3.2             2.6             2.2   
    1984              1.3             2.8             1.8             2.7   
    1985              1.1             1.8             2.6             2.8   
    1986              1.2             2.2             2.7             3.2   
...                   ...             ...             ...             ...   
WSB 1993              2.2             2.8             2.5             1.3   
    1994              1.5             3.2             2.4             2.0   
    1995              1.6             3.1             2.8             0.9   
    1996              1.9             3.7             1.9             1.8   
    1997              1.6             3.2             3.3             3.0   

           tov_per_game_SG pts_per_game_C pts_per_game_PF pts_per_game_PG  \
tm  season                                                                  
ATL 1982               2.7            6.1            18.6            10.5   
    1983               2.6            7.8            19.0            12.6   
    1984               2.6            8.6            18.9            12.3   
    1985               3.3            6.3             9.8            14.1   
    1986               1.4            5.6            12.3            11.5   
...                    ...            ...             ...             ...   
WSB 1993               1.5           17.4            14.7            14.8   
    1994               2.0            6.6            17.1            12.1   
    1995               1.9           10.0            20.1            13.0   
    1996               1.0           14.5            22.1            10.0   
    1997               1.2           10.6            20.1            17.2   

           pts_per_game_SF pts_per_game_SG  
tm  season                                  
ATL 1982              18.5            17.8  
    1983              17.5            16.0  
    1984              21.6            13.2  
    1985              27.4            16.3  
    1986              30.3            12.9  
...                    ...             ...  
WSB 1993              18.6             9.3  
    1994              18.2            18.2  
    1995               3.9            16.6  
    1996              15.1             8.6  
    1997              19.1            10.6  

[1184 rows x 55 columns]

In [34]:
print(X.columns) # now we can look through this, and see that all of these will be numeric
# any non-numeric columns would cause issues.

Index(['g_C', 'g_PF', 'g_PG', 'g_SF', 'g_SG', 'gs_C', 'gs_PF', 'gs_PG',
       'gs_SF', 'gs_SG', 'fg_percent_C', 'fg_percent_PF', 'fg_percent_PG',
       'fg_percent_SF', 'fg_percent_SG', 'x3p_percent_C', 'x3p_percent_PF',
       'x3p_percent_PG', 'x3p_percent_SF', 'x3p_percent_SG', 'e_fg_percent_C',
       'e_fg_percent_PF', 'e_fg_percent_PG', 'e_fg_percent_SF',
       'e_fg_percent_SG', 'trb_per_game_C', 'trb_per_game_PF',
       'trb_per_game_PG', 'trb_per_game_SF', 'trb_per_game_SG',
       'ast_per_game_C', 'ast_per_game_PF', 'ast_per_game_PG',
       'ast_per_game_SF', 'ast_per_game_SG', 'stl_per_game_C',
       'stl_per_game_PF', 'stl_per_game_PG', 'stl_per_game_SF',
       'stl_per_game_SG', 'blk_per_game_C', 'blk_per_game_PF',
       'blk_per_game_PG', 'blk_per_game_SF', 'blk_per_game_SG',
       'tov_per_game_C', 'tov_per_game_PF', 'tov_per_game_PG',
       'tov_per_game_SF', 'tov_per_game_SG', 'pts_per_game_C',
       'pts_per_game_PF', 'pts_per_game_PG', 'pts_per_game_SF',


In [35]:
# train-test split using sklearn:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")
# sports data is inherintly quite noisy, so this does show a pretty decent corrolation, which is great!

Mean Squared Error: 60.22159330503293
R^2 Score: 0.6184185169508081


In [36]:
# lets look at a feasible example:
# we can look how we compare to the actual results from last year!

# we can query the db for this!
last_year_data = cleaned_1[cleaned_1['season'] == 2024] # only want from last season [2024]

ly_pivoted = last_year_data.pivot(index=['tm', 'season'], columns='pos', values=['g','gs','pos', 'fg_percent','x3p_percent','e_fg_percent','trb_per_game','ast_per_game','stl_per_game','blk_per_game','tov_per_game','pts_per_game'])

# Flatten the multi-index columns
ly_pivoted.columns = ['_'.join(col).strip() for col in ly_pivoted.columns]

ly_pivoted['w'] = last_year_data.groupby(['tm', 'season'])['w'].first().values

test2024 = ly_pivoted.drop(columns=['w', 'pos_PG', 'pos_SG', 'pos_SF', 'pos_PF', 'pos_C'])
correct_ans = ly_pivoted['w']
# run through model
pred_2024 = model.predict(test2024)

# for easy comparing, lets look at the resulting df:
result_df = pd.DataFrame({'correct': correct_ans, 'predictions': pred_2024})
result_df


,,correct,predictions
tm,season,,
ATL,2024,36.0,41.325999
BOS,2024,64.0,58.029149
BRK,2024,32.0,41.534551
CHI,2024,39.0,52.490533
CHO,2024,21.0,38.560771
CLE,2024,48.0,56.231078
DAL,2024,50.0,50.190966
DEN,2024,57.0,65.335810
DET,2024,14.0,34.541542


In [37]:
# For the most part, The model preformed very well! There was only a few large errors. 
# This means that the stats I have chosen can be somewhat reliably used to predict wins in the NBA.

# Now I want to use everything I have to predict the current season!

# There are some issues:
# I cannot just use the stats so far this season. I used games played and games started
# as features in the win prediction formula, and since the season is early,
# the per game stats still may not represent how they will remain for the season
# (players can start the season better than is mantainable, or worse)
# So for each starter, I will take their statistical preformace from last year instead, and use that
# For all starters who do not have a statline for last year (rookie players), I will extend
# their per game stats for the season (imperfect, but still okay), and then I will simulate with those numbers
# extending already starting rookies to start 50 games in the year and play in 60 of them.
# This will not account for young players improving, or old players regressing. 
# My idea is that accounting for neither will on average keep things unweighted in either direction. 
# The code for this:

this_year_data = cleaned_1[cleaned_1['season'] == 2025] # only want for this season
# we can pull from player_stats db 

updated_stats = []
# Loop over all players in this year's data
for player_id in this_year_data['player_id']:
    # Query to get player's stats from last year
    query = f"SELECT * FROM player_stats WHERE player_id = {player_id} AND season = 2024;"
    last_year_stats = pd.read_sql_query(query, db)
    
    # Get the relevant info from this year's data for the player
    current_year_row = this_year_data[this_year_data['player_id'] == player_id].iloc[0]
    
    
    additional_info = current_year_row[['player_id', 'tm', 'season', 'pos']]
    
    if not last_year_stats.empty:
        # Use last year's stats, add columns from this year
        last_year_stats_row = last_year_stats.iloc[0]
        row_better = last_year_stats_row.drop(['tm', 'season', 'player_id', 'pos'])
        combined_stats = pd.concat([additional_info, pd.Series(row_better)], axis=0)
    else:
        # No data for last year (rookie), extend this year's stats
        
        current_year_row['g'] = 50
        current_year_row['gs'] = 65
        combined_stats = current_year_row
    
    updated_stats.append(combined_stats)

# Convert the updated stats back to a DataFrame
updated_stats_df = pd.DataFrame(updated_stats)
updated_stats_df = updated_stats_df.reset_index().drop(columns=['index', 'w'])
updated_stats_df

,player_id,tm,season,pos,player,g,gs,fg_percent,x3p_percent,e_fg_percent,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game
0,5041.0,ATL,2025,PF,David Roddy,48.0,13.0,0.402,0.301,0.462,4.2,1.6,0.5,0.2,1.1,8.4
1,4230.0,ATL,2025,C,Clint Capela,73.0,73.0,0.571,0.000,0.571,10.6,1.2,0.6,1.5,1.0,11.5
2,4675.0,ATL,2025,PG,Trae Young,54.0,54.0,0.430,0.373,0.516,2.8,10.8,1.3,0.2,4.4,25.7
3,5046.0,ATL,2025,SG,Dyson Daniels,61.0,16.0,0.447,0.311,0.514,3.9,2.7,1.4,0.4,1.0,5.8
4,4949.0,ATL,2025,SF,Jalen Johnson,56.0,52.0,0.511,0.355,0.562,8.7,3.6,1.2,0.8,1.8,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,4534.0,WAS,2025,PF,Kyle Kuzma,70.0,70.0,0.463,0.336,0.521,6.6,4.2,0.5,0.7,2.7,22.2
146,5215.0,WAS,2025,C,Alex Sarr,50.0,65.0,0.376,0.244,0.429,6.1,2.1,0.7,1.8,1.3,10.6
147,5223.0,WAS,2025,PG,Carlton Carrington,50.0,65.0,0.401,0.352,0.489,3.9,4.4,1.2,0.3,1.9,8.8
148,4737.0,WAS,2025,SG,Jordan Poole,78.0,66.0,0.413,0.326,0.491,2.7,4.4,1.1,0.3,2.4,17.4


In [38]:
# Great! Now all this data is ready to be used to predict the future. We can reuse the process from before:
# we can query the db for this!
# updated_stats_df

duplicates = updated_stats_df[['tm', 'season', 'pos']].duplicated()
if duplicates.any():
    print("Duplicates found:")
    print(updated_stats_df[duplicates])

df_2025 = updated_stats_df.pivot(index=['tm', 'season'], columns='pos', values=['g','gs','pos', 'fg_percent','x3p_percent','e_fg_percent','trb_per_game','ast_per_game','stl_per_game','blk_per_game','tov_per_game','pts_per_game'])

# Flatten the multi-index columns
df_2025.columns = ['_'.join(col).strip() for col in df_2025.columns]


test2025 = df_2025.drop(columns=['pos_PG', 'pos_SG', 'pos_SF', 'pos_PF', 'pos_C'])
# run through model
pred_2025 = model.predict(test2025)

results_2025 = df_2025.reset_index()

result_df = pd.DataFrame({'team': results_2025['tm'], 'predictions': pred_2025})
result_df




,team,predictions
0,ATL,38.155056
1,BOS,55.265162
2,BRK,31.136112
3,CHI,45.820074
4,CHO,20.847416
5,CLE,54.766466
6,DAL,49.528550
7,DEN,57.346490
8,DET,37.405747
9,GSW,44.815830


In [39]:
# Get the model's coefficients and intercept
coefficients = model.coef_
intercept = model.intercept_

# Get the feature names (columns)
features = X_train.columns

coefficients = model.coef_
features = X.columns

# Create a DataFrame to pair features with their coefficients
coefficients_df = pd.DataFrame({
    'Feature': features,
    'Coefficient': coefficients
})

coefficients_df
# we can observe some intersting trends looking at the coefficients. I will explain more in my video & writeup.

,Feature,Coefficient
0,g_C,0.059691
1,g_PF,0.063013
2,g_PG,-0.013457
3,g_SF,-0.039852
4,g_SG,0.008802
5,gs_C,0.089840
6,gs_PF,0.044252
7,gs_PG,0.133344
8,gs_SF,0.080300
9,gs_SG,0.088621


In [41]:
db.close() # cant forget to close it